In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import glob
from scipy.ndimage.filters import gaussian_filter
from astropy.io import fits
from astropy.wcs import WCS
import healpy as hp
import reproject

In [3]:
from __future__ import unicode_literals

figsize(8,6)

matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.unicode'] = True
    
rcParams['xtick.labelsize'] = 20
rcParams['xtick.major.size'] = 10
rcParams['xtick.major.width'] = 1
rcParams['xtick.major.pad'] = 10
rcParams['xtick.minor.size'] = 5
rcParams['xtick.minor.width'] = 1

rcParams['ytick.labelsize'] = 20
rcParams['ytick.major.size'] = 10
rcParams['ytick.major.width'] = 1
rcParams['ytick.major.pad'] = 10
rcParams['ytick.minor.size'] = 5
rcParams['ytick.minor.width'] = 1

rcParams['legend.fontsize'] = 12
rcParams['legend.borderaxespad'] = 2

rcParams['axes.labelsize'] = 24
rcParams['axes.titlesize'] = 24

In [4]:
lonc = 107.2
latc = 5.2
lonra = [105.6, 108.8]
latra = [3.6, 6.8]
d2r = np.pi / 180.

In [5]:
def plot_destriped(bank='A', removeplane=True, sbeam=0., nside=4096):
    imfile = fits.open('destripingdata_bank'+bank+'_session5_weights1.fits')

    if bank == 'A':
        cfreq = 4.575
    elif bank == 'B':
        cfreq = 5.625
    elif bank == 'C':
        cfreq = 6.125
    elif bank == 'D':
        cfreq = 7.175
    beam = 12.6 / cfreq

    pixbeam = 1. 
    beamarea = np.pi / (4. * np.log(2)) * (beam / 60. * d2r)**2
    units = 1. / beamarea

    newbeam = np.sqrt(beam**2 - pixbeam**2)
    sigma = newbeam / pixbeam / (2. * np.sqrt(2. * np.log(2)))

    mdata = imfile[0].data
    datamap = gaussian_filter(mdata, sigma) * units 

    hitmask = imfile[0].data != 0
    hits = np.zeros_like(imfile[0].data)
    hits[hitmask] = 1.
    shits = gaussian_filter(hits, sigma)

    if removeplane:
        X = np.arange(180)
        RX, RY = np.meshgrid(X, X)
        radius = np.sqrt((RX-90)**2 + (RY-90)**2)
        rmask = radius < 25

        XX = RX.flatten()
        YY = RY.flatten()
        Z = datamap.flatten()
        mask = Z == 0 
        Z[mask] = np.nan
        masks = ~np.isnan(Z) * ~rmask.flatten()
        data = np.c_[XX[masks], YY[masks], Z[masks]]
        data2 = np.c_[XX, YY, Z]
        A = np.c_[data[:, 0], data[:, 1], np.ones(data.shape[0])]
        C, _, _, _ = linalg.lstsq(A, data[:, 2])
        res = C[0] * RX + C[1] * RY + C[2]
        datamap -= res

    if sbeam > 0:
        newbeam = np.sqrt(sbeam**2 - beam**2)
        sigma = newbeam / pixbeam / (2. * np.sqrt(2. * np.log(2)))
        datamap = gaussian_filter(datamap, sigma) 

    mask = shits < 0.3
    datamap[mask] = np.nan
    imfile[0].data = datamap
    hpimage, footprint = reproject.reproject_to_healpix(imfile, 'galactic', nside=nside)
    return hpimage

In [6]:
bankA = plot_destriped('A', sbeam=40.)

In [7]:
bankB = plot_destriped('B', sbeam=40.)

In [8]:
bankC = plot_destriped('C', sbeam=40.)

In [15]:
bankD = plot_destriped('D', sbeam=40.)

In [9]:
nside = hp.get_nside(bankA)
vecc = hp.rotator.dir2vec(lonc, latc, lonlat=True)
rmask = hp.query_disc(nside, vecc, (45./60.)*d2r)
amaskout = set(hp.query_disc(nside, vecc, (80./60.)*d2r))
amaskin = set(hp.query_disc(nside, vecc, (60./60.)*d2r))
amask = np.array(list(amaskout.difference(amaskin)))

In [20]:
xdata = bankA * hp.nside2pixarea(4096)
xdata -= np.nanmedian(xdata[amask])
rdata = xdata[rmask]
adata = xdata[amask]
rms = np.nanstd(adata) * np.sqrt(len(rdata) + pi/2. * float(len(rdata)**2) / len(adata))
print np.nansum(rdata)
print rms

18.0860055748
0.0672948359813


In [21]:
xdata = bankB * hp.nside2pixarea(4096)
xdata -= np.nanmedian(xdata[amask])
rdata = xdata[rmask]
adata = xdata[amask]
rms = np.nanstd(adata) * np.sqrt(len(rdata) + pi/2. * float(len(rdata)**2) / len(adata))
print np.nansum(rdata)
print rms

17.5061433323
0.0810618422286


In [22]:
xdata = bankC * hp.nside2pixarea(4096)
xdata -= np.nanmedian(xdata[amask])
rdata = xdata[rmask]
adata = xdata[amask]
rms = np.nanstd(adata) * np.sqrt(len(rdata) + pi/2. * float(len(rdata)**2) / len(adata))
print np.nansum(rdata)
print rms

17.7466661872
0.123176390199


In [23]:
xdata = bankD * hp.nside2pixarea(4096)
xdata -= np.nanmedian(xdata[amask])
rdata = xdata[rmask]
adata = xdata[amask]
rms = np.nanstd(adata) * np.sqrt(len(rdata) + pi/2. * float(len(rdata)**2) / len(adata))
print np.nansum(rdata)
print rms

32.3860771746
0.325624019512
